# Import all the libraries and packages

In [ ]:
#!pip install eemont

In [1]:
import os
import ee
import eemont
import geemap
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
ee.Initialize()

In [2]:
# Test if geemap is activated
m = geemap.Map()
# m

# Calculate a collected NDWI Map

In [3]:
band_l8 = ['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7', 'QA_PIXEL']
band_l5_l7 = ['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B7', 'QA_PIXEL']
l8 = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2")
l7 = ee.ImageCollection("LANDSAT/LE07/C02/T1_L2")
l5 = ee.ImageCollection("LANDSAT/LT05/C02/T1_L2")
l8_bands = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2").select(band_l8)
l7_bands = ee.ImageCollection("LANDSAT/LE07/C02/T1_L2").select(band_l5_l7)
l5_bands = ee.ImageCollection("LANDSAT/LT05/C02/T1_L2").select(band_l5_l7)
# Import Algonquin Park Boundary Shapefile for clipping and fitlering the image collection
Algonquin_Boundary = ee.FeatureCollection("projects/ee-wang25/assets/Algonquin_Boundary").geometry()

#if we also intend to filter the image caputre day of year, use "ee.Filter.dayOfYear(91,250)"
filters = [
    ee.Filter.intersects('.geo', Algonquin_Boundary),
]

l8_bands_filtered = l8_bands.filter(filters)
l7_bands_filtered = l7_bands.filter(filters)
l5_bands_filtered = l5_bands.filter(filters)

def getBit(n):
    # Returns a GEE server-side object representing `int(2^n)`
    return ee.Number(2).pow(n).int()


def addMaskBand(image):   
    qa = image.select("QA_PIXEL")
    
    dilatedCloudBit = getBit(1)
    cirrusBit = getBit(2)
    cloudBit = getBit(3)
    cloudShadowBit = getBit(4)
    snowBit = getBit(5)
    
    # Define the mask by extracting these bits and reclassifying the pixel based on the bit's value
    mask = ee.Image(0)\
        .where(qa.bitwiseAnd(dilatedCloudBit).neq(0), 1)\
        .where(qa.bitwiseAnd(cloudBit).neq(0), 2)\
        .where(qa.bitwiseAnd(cirrusBit).neq(0), 3)\
        .where(qa.bitwiseAnd(cloudShadowBit).neq(0), 4)\
        .where(qa.bitwiseAnd(snowBit).neq(0), 5)\
        .updateMask(image.select('QA_PIXEL').mask())\
        .rename("cloud_shadow_snow_mask")
    
    # return original image with this mask added as an extra band
    return image.addBands(mask)

l8_bands_filtered_masked = ee.ImageCollection(l8_bands_filtered).map(addMaskBand)
l7_bands_filtered_masked = ee.ImageCollection(l7_bands_filtered).map(addMaskBand)
l5_bands_filtered_masked = ee.ImageCollection(l5_bands_filtered).map(addMaskBand)

def maskImage(image):
    cloud_shadow_snow = image.select("cloud_shadow_snow_mask")
    return image.updateMask(cloud_shadow_snow.eq(0))

l8_final = l8_bands_filtered_masked.map(maskImage)
l7_final = l7_bands_filtered_masked.map(maskImage)
l5_final = l5_bands_filtered_masked.map(maskImage)

l8_vizParams = {
    "bands": ['SR_B5', 'SR_B4', 'SR_B3'],
    "min": [0, 0, 0],
    "max": [30000, 20000, 20000]
}

l5_l7_vizParams = {
    "bands": ['SR_B4', 'SR_B3', 'SR_B2'],
    "min": [0, 0, 0],
    "max": [30000, 20000, 20000]
}

#---------------------------------------EVERYTHING UP TO HERE IS THE SAME FOR ALL INDICES---------------------------


def calcNDWI(image):
    return image.normalizedDifference(['SR_B2','SR_B4']).multiply(10000).rename("NDWI").set({'system:time_start': image.get('system:time_start')})

l5_NDWI = l5_final.map(calcNDWI)
l7_NDWI = l7_final.map(calcNDWI)

ndwiParams = {
  'min': -5000,
  'max': 5000,
  'palette': ['#762a83','#9970ab','#c2a5cf','#e7d4e8','#f7f7f7','#d9f0d3','#a6dba0','#5aae61','#1b7837']
}

#calcNDWI is redefined here to change the bands used for calculating NDWI for Landsat 8
def calcNDWI(image):
    return image.normalizedDifference(['SR_B3','SR_B5']).multiply(10000).rename("NDWI").set({'system:time_start': image.get('system:time_start')})

l8_NDWI = l8_final.map(calcNDWI)

l8_ndwiParams = {
  'min': -5000,
  'max': 5000,
  'palette': ['#762a83','#9970ab','#c2a5cf','#e7d4e8','#f7f7f7','#d9f0d3','#a6dba0','#5aae61','#1b7837']
}

# Merge l5_NDWI and l7_NDWI
landsat_5_7_NDWI = l5_NDWI.merge(l7_NDWI)
# Merge landsat_5_7_NDWI with l8_NDWI
landsat_5_7_8_NDWI = landsat_5_7_NDWI.merge(l8_NDWI)

m = geemap.Map()
m.addLayer(landsat_5_7_8_NDWI, ndwiParams, "l5_7_8_NDWI")
m.setCenter(-78.3, 45.77, 9)
m

Map(center=[45.77, -78.3], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(childre…

# Using Geopandas to Loop through Shapefile and Export NDWI Value

In [6]:
import geopandas as gpd

# Use ArcGIS Pro to modify this file or to select more pixels
# ArcGIS Pro: Upperstream Location
# https://www.lioapplications.lrc.gov.on.ca/OWIT/index.html?viewer=OWIT.OWIT&locale=en-CA
upstream = gpd.read_file("D:/Yannans Stuff/Data/Visible Dam/Visible Dam.shp")
upstream

,Id,geometry
0,0,POINT (-78.68041 45.69172)
1,1,POINT (-78.67734 45.69231)
2,2,POINT (-78.67780 45.68798)


In [7]:
for i in upstream.index:
    print(upstream.loc[i,'Id'])

0
1
2


In [8]:
# outdir = "D:/Yannans Stuff/Data/NDWI_Export" # specify an output directory
# os.makedirs(outdir, exist_ok = True) ## create it if it doesn't already exist

In [9]:
for i in upstream.index:
    # Here we will convert each individual point into a ee.geometry
    gpd_geom = upstream.loc[i,'geometry']
    x = gpd_geom.xy[0][0]
    y = gpd_geom.xy[1][0]
    ee_geom = ee.Geometry.Point([x,y])
    
    # Followed by the eemont code
    ts = landsat_5_7_8_NDWI.getTimeSeriesByRegion(geometry = ee_geom,
                               reducer = ee.Reducer.mean(),
                               scale = 30)
    tsPandas = geemap.ee_to_pandas(ts)
    tsPandas[tsPandas == -9999] = np.nan
    tsPandas = tsPandas.dropna()
    
    #Create an output label using the output directory and the 'id' column
    ID = upstream.loc[i,'Id']
    tsPandas.to_csv(r"D:\Yannans Stuff\Data\Visible Dam\Landsat5_7_8_NDWI_TS_{0}.csv".format(ID))